<a href="https://colab.research.google.com/github/barroso2501/GIS_MapBiomas/blob/main/mosaico_tif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instale as bibliotecas necessárias
!pip install numpy gdal

import os
import numpy as np
from google.colab import drive
from osgeo import gdal

# Monte o Google Drive
drive.mount('/content/drive')

# Defina o caminho para a pasta onde estão os arquivos TIFF
tif_folder = '/content/drive/MyDrive/Mosaic/'

# Verifique se o diretório existe
if not os.path.exists(tif_folder):
    raise RuntimeError(f"A pasta {tif_folder} não existe. Verifique o caminho.")

# Liste os arquivos .tif na pasta
tif_files = [os.path.join(tif_folder, f) for f in os.listdir(tif_folder) if f.endswith('.tif')]

# Verifique se encontrou arquivos TIFF
if len(tif_files) == 0:
    raise RuntimeError(f"Nenhum arquivo TIFF foi encontrado na pasta {tif_folder}. Verifique os arquivos.")

print(f'Arquivos TIFF encontrados: {tif_files}')

# Processamento em blocos usando GDAL, mantendo o maior valor nas áreas de sobreposição
def process_in_blocks(input_files, output_tif, block_size=256):
    # Certifique-se de que os arquivos foram passados corretamente
    if not input_files or len(input_files) == 0:
        raise RuntimeError("Nenhum arquivo de entrada fornecido para o VRT.")

    # Verifique se os arquivos de entrada são strings válidas
    for file in input_files:
        if not isinstance(file, str):
            raise RuntimeError(f"O arquivo {file} não é uma string válida.")

    # Criar o VRT e salvar como arquivo físico
    vrt_file = '/content/drive/MyDrive/Mosaic/temp.vrt'
    vrt_options = gdal.BuildVRTOptions(resampleAlg='near', separate=False)

    try:
        # Criar o VRT no disco
        vrt = gdal.BuildVRT(vrt_file, input_files, options=vrt_options)
        vrt.FlushCache()  # Garante que o VRT seja salvo corretamente
    except Exception as e:
        raise RuntimeError(f"Erro ao construir o VRT: {e}")

    if vrt is None:
        raise RuntimeError("Erro ao construir o VRT. Verifique os arquivos de entrada.")

    # Abrir o VRT recém-criado com GDAL
    src = gdal.Open(vrt_file)
    if src is None:
        raise RuntimeError("Erro ao abrir o VRT físico.")

    # Criar o arquivo TIFF de saída como 8-bit unsigned
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.Create(output_tif, src.RasterXSize, src.RasterYSize, 1, gdal.GDT_Byte)
    out_ds.SetGeoTransform(src.GetGeoTransform())
    out_ds.SetProjection(src.GetProjection())

    # Processar a imagem em blocos, mantendo o maior valor nas áreas de sobreposição
    for i in range(0, src.RasterYSize, block_size):
        for j in range(0, src.RasterXSize, block_size):
            x_block_size = min(block_size, src.RasterXSize - j)
            y_block_size = min(block_size, src.RasterYSize - i)

            # Inicializar uma matriz de bloco vazia para armazenar o valor máximo
            max_block = np.full((y_block_size, x_block_size), 0, dtype=np.uint8)

            # Comparar blocos de todas as imagens e pegar o valor máximo
            for tif_file in input_files:
                # Tente abrir o arquivo raster
                src_raster = gdal.Open(tif_file)
                if src_raster is None:
                    print(f"Erro ao abrir o arquivo TIFF: {tif_file}")
                    continue  # Pular para o próximo arquivo se não puder ser aberto

                try:
                    block = src_raster.ReadAsArray(j, i, x_block_size, y_block_size)
                    if block is None:
                        print(f"Erro ao ler o bloco de {tif_file}.")
                        continue
                    max_block = np.maximum(max_block, block)
                except Exception as e:
                    print(f"Erro ao processar o bloco do arquivo {tif_file}: {e}")
                    continue

            # Escrever o bloco resultante no arquivo de saída
            out_ds.GetRasterBand(1).WriteArray(max_block, j, i)

    # Fechar os arquivos
    out_ds.FlushCache()  # Garante que tudo seja salvo
    out_ds = None
    src = None
    vrt = None

# Caminho do arquivo TIFF de saída
output_tif = '/content/drive/MyDrive/Mosaic/mosaic_output.tif'

# Realize o processamento em blocos e crie o mosaico
process_in_blocks(tif_files, output_tif)

Streaming output truncated to the last 5000 lines.
Erro ao ler o bloco de /content/drive/MyDrive/Mosaic/raster5.tif.
Erro ao ler o bloco de /content/drive/MyDrive/Mosaic/raster6.tif.
Erro ao ler o bloco de /content/drive/MyDrive/Mosaic/raster1.tif.
Erro ao ler o bloco de /content/drive/MyDrive/Mosaic/raster2.tif.
Erro ao ler o bloco de /content/drive/MyDrive/Mosaic/raster3.tif.
Erro ao ler o bloco de /content/drive/MyDrive/Mosaic/raster4.tif.
Erro ao ler o bloco de /content/drive/MyDrive/Mosaic/raster5.tif.
Erro ao ler o bloco de /content/drive/MyDrive/Mosaic/raster6.tif.
Erro ao ler o bloco de /content/drive/MyDrive/Mosaic/raster1.tif.
Erro ao ler o bloco de /content/drive/MyDrive/Mosaic/raster2.tif.
Erro ao ler o bloco de /content/drive/MyDrive/Mosaic/raster3.tif.
Erro ao ler o bloco de /content/drive/MyDrive/Mosaic/raster4.tif.
Erro ao ler o bloco de /content/drive/MyDrive/Mosaic/raster5.tif.
Erro ao ler o bloco de /content/drive/MyDrive/Mosaic/raster6.tif.
Erro ao ler o bloco de /c